# Kaggle Competition:  Iceberg or Ship

## Notebook 3
## run resnext50 architecture

In [1]:
!ls -l

total 1812
-rw-rw-r-- 1 nidhin nidhin 623462 Jan 13 22:48 1_getdata.ipynb
-rw-rw-r-- 1 nidhin nidhin 245619 Jan 13 21:50 2_model_resnet34.ipynb
-rw-rw-r-- 1 nidhin nidhin  13184 Jan 13 23:37 3_model_resnext50.ipynb
-rw-rw-r-- 1 nidhin nidhin  64324 Jan 14 00:24 4_resnext_resize.ipynb
-rw-rw-r-- 1 nidhin nidhin 137540 Jan 13 21:44 baseline.ipynb
-rw-rw-r-- 1 nidhin nidhin    536 Jan 13 19:15 helper.py
-rw-rw-r-- 1 nidhin nidhin 747405 Jan 13 22:32 other_groverpr.ipynb
drwxrwxr-x 2 nidhin nidhin   4096 Jan 13 19:15 __pycache__
-rw-rw-r-- 1 nidhin nidhin    185 Jan 13 17:48 README.md


In [2]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.imports import *

In [4]:
# This file contains all the main external libs we'll use
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
import pandas as pd
import numpy as np
import shutil
from IPython.display import FileLink, FileLinks

from sklearn.model_selection import train_test_split
import helper

In [5]:
#PATH = "/home/nidhin/data/iceberg/"
PATH = os.path.expanduser("~/data/iceberg/")

In [6]:
!ls -l {PATH}

total 1975500
drwxrwxr-x  6 nidhin nidhin       4096 Jan 13 21:37 composites
drwxrwxr-x  2 nidhin nidhin       4096 Jan 13 19:06 models
-rw-rw-r--  1 nidhin nidhin     117951 Oct 23 17:27 sample_submission.csv
-rw-rw-r--  1 nidhin nidhin      38566 Jan  7 20:28 sample_submission.csv.7z
drwxrwxr-x  3 nidhin nidhin       4096 Jan 13 21:20 sub
drwxrwxr-x  2 nidhin nidhin     286720 Jan 13 22:48 test
-rw-rw-r--  1 nidhin nidhin 1521771850 Oct 23 17:27 test.json
-rw-rw-r--  1 nidhin nidhin  257127394 Jan  7 20:28 test.json.7z
drwxrwxr-x 15 nidhin nidhin       4096 Jan 13 21:50 tmp
drwxrwxr-x  4 nidhin nidhin       4096 Jan 13 22:48 train
-rw-rw-r--  1 nidhin nidhin  196313674 Oct 23 17:23 train.json
-rw-rw-r--  1 nidhin nidhin   44932785 Jan  7 20:28 train.json.7z
-rw-rw-r--  1 nidhin nidhin    2275916 Jan  8 01:44 train_modified.zip
drwxrwxr-x  4 nidhin nidhin       4096 Jan 13 22:48 valid


In [7]:
sz=299
arch=resnext50
bs=20

In [8]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs,test_name='test',)
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [9]:
learn.fit(1e-2, 8)
learn.precompute=False

HBox(children=(IntProgress(value=0, description='Epoch', max=8), HTML(value='')))

[ 0.       0.63076  0.5576   0.75   ]                      
[ 1.       0.58425  0.42959  0.78824]                       
[ 2.       0.58511  0.4956   0.78529]                       
[ 3.       0.49668  0.45511  0.78824]                       
[ 4.       0.49822  0.44512  0.81176]                       
[ 5.       0.46349  0.47785  0.78824]                       
[ 6.       0.42828  0.43138  0.81471]                       
[ 7.       0.45766  0.50244  0.8    ]                       



In [10]:
learn.fit(1e-2, 4, cycle_len=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=8), HTML(value='')))

[ 0.       0.74512  0.52371  0.73823]                      
[ 1.       0.55291  0.42364  0.79118]                      
[ 2.       0.53689  0.43012  0.79118]                      
[ 3.       0.48201  0.55008  0.76177]                      
[ 4.       0.49263  0.48672  0.77647]                      
[ 5.       0.43207  0.49802  0.76471]                      
[ 6.       0.45387  0.42934  0.78824]                      
[ 7.       0.46766  0.41014  0.8    ]                      



In [11]:
learn.unfreeze()
lr=np.array([1e-4,1e-3,1e-2])

In [12]:
learn.fit(lr, 3, cycle_len=3,)

HBox(children=(IntProgress(value=0, description='Epoch', max=9), HTML(value='')))

[ 0.       0.59807  0.52486  0.78529]                      
[ 1.       0.45908  0.35837  0.82353]                      
[ 2.       0.37501  0.29855  0.87353]                      
[ 3.       0.42569  0.35269  0.82647]                      
[ 4.       0.34703  0.30805  0.86177]                      
[ 5.       0.29847  0.31136  0.86177]                      
[ 6.       0.28618  0.28742  0.87941]                      
[ 7.       0.26019  0.28913  0.89118]                      
[ 8.       0.22024  0.32283  0.85882]                      



In [13]:
learn.save('224_all_50')

In [14]:
learn.load('224_all_50')

In [15]:
log_preds,y = learn.TTA()
preds = np.mean(np.exp(log_preds),axis=0)
(metrics.log_loss(y,preds), accuracy(preds,y))

(0.2914880899536279, 0.87850467289719625)

In [16]:
log_preds,y = learn.TTA(is_test=True)

In [17]:
test_df = pd.read_json(PATH+"/test.json")
helper.make_test_file(log_preds,test_df,model_name='resnext50_sz299_bs20_colorscale',
                      classes=data.classes,
                      submission_folder =f'{PATH}sub'
                     )

Saving to /home/nidhin/data/iceberg/sub/resnext50_sz299_bs20_colorscalesubm.gz
